In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.getcwd()) 

X = pd.read_csv('X_features.csv', index_col=0)
y = pd.read_csv('y_features.csv', index_col=0)
groups = X.pop('Feature_7')

test = pd.read_csv('test_features.csv', index_col=0).drop(columns = 'Feature_7')
sample = pd.read_csv('sample_submission_2.csv')
df_train = pd.read_csv('train.csv').set_index('Id')
df_test = pd.read_csv('test_2.csv').set_index('Id')
weights = df_train.loc[:,'Weight_Intraday':'Weight_Daily']

C:\Users\matze\Documents\Winton\CRISPDM


In [15]:
#Helper
def typecasting(df):
    categorial_cols = ['Feature_20','Feature_5']
    binary_cols = ['Feature_16']

    for col in categorial_cols:
        df[col] = df[col].astype('category')
    
    for col in binary_cols:
        df[col] = (df[col]-1).astype('category')

def trainscore(y_true,y_pred,weights):
    weights = weights.loc[y_true.index,:]
    daily = pd.concat([weights.Weight_Daily]*2,axis=1)
    minute = pd.concat([weights.Weight_Intraday]*60,axis=1)
    weights = pd.concat([minute,daily],axis=1)
    weights.columns = df_train.loc[:,'Ret_121':'Ret_PlusTwo'].columns
    
    
    wmea = (((abs(y_true-y_pred))*weights).values.flatten().sum()/y_true.size)
    
    
    #minute_ret = abs(y_true.loc[:,'Ret_121':'180']-y_pred.loc[:,'Ret_121':'Ret_180'])*weights.Weight_Intraday
    return wmea


def create_core_features(train,test):
    X_train=train.loc[:,'Feature_1':'Ret_MinusOne']
    X_val=test.loc[:,'Feature_1':'Ret_MinusOne']
    from sklearn.preprocessing import Imputer

    typecasting(X_train)
    X_train = pd.get_dummies(X_train,drop_first=True)

    typecasting(X_val)
    X_val = pd.get_dummies(X_val,drop_first=True)

    imputer = Imputer(strategy='mean', axis=0)
    X_train = pd.DataFrame(imputer.fit_transform(X_train), index= X_train.index,columns=X_train.columns)
    X_val = pd.DataFrame(imputer.fit_transform(X_val), index= X_val.index,columns=X_val.columns)
    X_train['Ret_MinutePast'] = train.loc[:,'Ret_2':'Ret_120'].sum(axis=1)
    X_val['Ret_MinutePast'] = test.loc[:,'Ret_2':'Ret_120'].sum(axis=1)
    return X_train, X_val

### Creating featuresets

In [17]:
core_features, core_features_test = create_core_features(df_train,df_test)
# generated features is X
#Adding the interactions to Featureset
crafted_features = pd.read_csv('BA_Features/X_features_BA.csv', index_col=0)
crafted_features_test = pd.read_csv('BA_Features/test_features_BA.csv', index_col=0)

In [21]:
crafted_features['interaction_3']= crafted_features.PR_120 * crafted_features.grouped_mad_fet_7
crafted_features_test['interaction_3']= crafted_features_test.PR_120 * test.grouped_mad_fet_7

#X['interaction_4']= X.smoothed_minute_mean * X.minute_mad
#test['interaction_4']= test.smoothed_minute_mean * test.minute_mad

crafted_features['interaction_5']= crafted_features.minute_mad * crafted_features.PR_120
crafted_features_test['interaction_5']= crafted_features_test.minute_mad * crafted_features_test.PR_120

crafted_features['interaction_6']= crafted_features.smoothed_minute_mean * crafted_features.grouped_mad_fet_5
crafted_features_test['interaction_6']= crafted_features_test.smoothed_minute_mean * crafted_features_test.grouped_mad_fet_5

crafted_features['interaction_7']= crafted_features.PR_120 * crafted_features.fet_7_Minute_MAD
crafted_features_test['interaction_7']= crafted_features_test.PR_120 * crafted_features_test.fet_7_Minute_MAD

crafted_features['interaction_8']= crafted_features.fet_7_Minute_MAD * crafted_features.fet_7_Minute_Mean
crafted_features_test['interaction_8']= crafted_features_test.fet_7_Minute_MAD * crafted_features_test.fet_7_Minute_Mean

print(crafted_features.shape,crafted_features_test.shape)

(40000, 102) (120000, 102)


In [293]:
all_features = pd.concat([core_features,crafted_features],axis=1).drop('Feature_7',axis='columns')
all_features_test = pd.concat([core_features_test,crafted_features_test],axis=1).drop('Feature_7',axis='columns')

features_and_target = all_features.copy()
features_and_target['Ret_PlusOne'] = y.Ret_PlusOne
features_and_target['Ret_PlusTwo'] = y.Ret_PlusTwo
features_and_target['Ret_MinutesFut'] = df_train.loc[:,'Ret_121':'Ret_180'].sum(axis=1)



In [294]:
#import time
#t0=time.time()
#corr_spearman = features_and_target.corr(method='spearman').to_csv('BA_Features/spearman_cor.csv')
print("time:", round(time.time()-t0, 3), "s")
#corr_pearson = features_and_target.corr(method='pearson').to_csv('BA_Features/pearson_cor.csv')
#print("time:", round(time.time()-t0, 3), "s")

time: 111.148 s


In [ ]:
'''
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
n=20000

t0 = time.time()
mic_Ret_PlusOne = mutual_info_regression(all_features.sample(n,random_state=0), y.Ret_PlusOne.sample(n,random_state=0), random_state=0)
print('Time: ',time.time()-t0)
t0 = time.time()
#select10 = SelectKBest(score_func=mutual_info_regression, k=20).fit(all_features.sample(n,random_state=0), y.Ret_PlusOne.sample(n,random_state=0))
print('Time: ',time.time()-t0)
mic_Ret_PlusTwo = mutual_info_regression(all_features.sample(n,random_state=0), y.Ret_PlusTwo.sample(n,random_state=0), random_state=0)
print('Time: ',time.time()-t0)
t0 = time.time()
mic_Ret_Minute = mutual_info_regression(all_features.sample(n,random_state=0), df_train.loc[:,'Ret_121':'Ret_180'].sum(axis=1).sample(n,random_state=0), random_state=0)

print('Time: ',time.time()-t0)
'''

In [132]:
#pd.DataFrame([mic_Ret_PlusOne,mic_Ret_PlusTwo,mic_Ret_Minute], index = ['MIC_PlusOne','MIC_PlusTwo','MIC_Minute'],columns=all_features.columns.values).T.to_csv('BA_Features/MIC.csv')

In [ ]:
'''from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures()
X_poly = pd.read_csv('X_features_2.csv', index_col=0)
X_poly_test = pd.read_csv('test_features_2.csv', index_col=0)
X_poly = pd.DataFrame(poly.fit_transform(X_poly), columns= poly.get_feature_names(),index=X_poly.index).drop(['1'],axis='columns')
X_poly_test=pd.DataFrame(poly.fit_transform(X_poly_test), columns= poly.get_feature_names(),index=X_poly_test.index).drop(['1'],axis='columns')
display(X_poly.head())
display(X_poly_test.head())

X_poly_and_target = X_poly.copy()
X_poly_and_target['Ret_PlusOne'] = y.Ret_PlusOne
X_poly_and_target['Ret_PlusTwo'] = y.Ret_PlusTwo
X_poly_and_target['Ret_MinutesFut'] = df_train.loc[:,'Ret_121':'Ret_180'].sum(axis=1)

X_poly_and_target.corr(method='spearman').to_csv('BA_Features/poly_cor.csv')'''


In [133]:
MIC = pd.read_csv('BA_Features/MIC.csv', index_col=0)
corr_spearman = pd.read_csv('BA_Features/spearman_cor.csv',index_col=0).drop(['Ret_MinutesFut','Ret_PlusOne','Ret_PlusTwo'])
corr_poly = pd.read_csv('BA_Features/poly_cor.csv',index_col=0).drop(['Ret_MinutesFut','Ret_PlusOne','Ret_PlusTwo'])


In [481]:
MIC_poly = pd.read_csv('BA_Features/MIC_Poly.csv', index_col=0)

## Predicting Ret_PlusOne

### Featuresets (Needed once for each Target)

In [581]:
results = pd.DataFrame(columns=['algo','featureset','target','fitting_time','zerobenchmark','validationmark','mae','params'])

In [482]:
mic10 = MIC.MIC_PlusOne.sort_values(ascending=False)[0:10].index
mic20 = MIC.MIC_PlusOne.sort_values(ascending=False)[0:20].index
mic30 = MIC.MIC_PlusOne.sort_values(ascending=False)[0:30].index
corr10 = abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[0:10].index
corr20 = abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[0:20].index
corr30 = abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[0:30].index
core=core_features.drop('Feature_7',axis='columns').columns.values
poly15_1 = ['x1 x8','x3 x16', 'x5 x10', 'x5 x16', 'x5', 'x5 x6', 'x3 x6', 'x3 x5', 'x5 x11', 'x3 x10', 'x3^2', 'x2^2', 'x3', 'x3 x12']
poly15_2 = abs(corr_poly.Ret_PlusOne).sort_values(ascending=False)[0:15].index
micpoly15 = MIC_poly.MIC_PlusOne.sort_values(ascending=False)[0:15].index

### Ridge

In [ ]:
#RFE Params
rfe_learner = Ridge()
rfe_params ={'alpha':100,'normalize':True,'max_iter':700,'random_state':[0]}
rfe_learner = rfe_learner.set_params(**rfe_params)
rfe_target = 'Ret_PlusOne'

#RFE Fitting
rfe = RFE(rfe_learner,n_features_to_select=10, step=1, verbose=0)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe10 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=20, step=1, verbose=0)
rfe.fit(all_features, y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe20 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=30, step=1, verbose=0)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe30 = all_features.columns[rfe.get_support(indices=True)]

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE

#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = Ridge()
target= 'Ret_PlusOne'
param_grid= {'alpha': [20,19,18,17,16,15,12,11,10,8,6,4,2,1,0.5],'normalize':[True],'max_iter':[700],'random_state':[0]}


#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,rfe10,rfe20,rfe30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','rfe10','rfe20','rfe30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

         ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


### Huber

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import HuberRegressor
from sklearn.feature_selection import RFE

#Model_Params

learner = HuberRegressor()
target= 'Ret_PlusOne'
param_grid= {'alpha': [100,20,19,18,17,16,15,12,11,10],'max_iter':[700]}


#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

         ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


### Extra Tree Regressor

In [552]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)

from sklearn.ensemble import ExtraTreesRegressor


#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = ExtraTreesRegressor()
target= 'Ret_PlusOne'
param_grid= {'max_depth': [3],#tiefer
             
             'max_features': 'auto',#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_leaf': [5000], #höher
             #'min_samples_split': [],
             'n_estimators': [100,150],#mehr bedeuten dauert länger
             'n_jobs':[-1],
            'random_state':[0],
            'max_features':['sqrt']}


#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

     ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


Index(['core', 'mic10', 'mic20', 'mic30', 'corr10', 'corr20', 'corr30',
       'polycorr15_1', 'polycorr15_2', 'micpoly15'],
      dtype='object')

'core'

'mic10'

'mic20'

'mic30'

...

'corr20'

'corr30'

'polycorr15_1'

'polycorr15_2'

'micpoly15'

## GradientBoostingRegressor

In [520]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)

from sklearn.ensemble import GradientBoostingRegressor

#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = GradientBoostingRegressor()
target= 'Ret_PlusOne'
param_grid= {'max_depth': [3,5],#tiefer
             'max_features': ['auto'],#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_split': [4000, 5000],#kann wahrscheinlich weg
             'n_estimators': [200, 400],
            'random_state':[0]}#mehr bedeuten dauert länger

#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 
    display(fitting_time)

    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

    ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


Index(['core', 'mic10', 'mic20', 'mic30', 'corr10', 'corr20', 'corr30',
       'rfe10', 'rfe20', 'rfe30', 'polycorr15_1', 'polycorr15_2', 'micpoly15'],
      dtype='object')

'core'

1061.6622803211212

'mic10'

244.98215889930725

...

400.7375810146332

'polycorr15_2'

259.4381444454193

'micpoly15'

320.37823510169983

# Ret_PlusTwo

In [472]:
mic10 = MIC.MIC_PlusTwo.sort_values(ascending=False)[0:10].index
mic20 = MIC.MIC_PlusTwo.sort_values(ascending=False)[0:20].index
mic30 = MIC.MIC_PlusTwo.sort_values(ascending=False)[0:30].index
corr10 = abs(corr_spearman.Ret_PlusTwo).sort_values(ascending=False)[0:10].index
corr20 = abs(corr_spearman.Ret_PlusTwo).sort_values(ascending=False)[0:20].index
corr30 = abs(corr_spearman.Ret_PlusTwo).sort_values(ascending=False)[0:30].index
#corrPear = abs(corr_pearson.Ret_PlusTwo).sort_values(ascending=False)[0:20].index
core=core_features.drop('Feature_7',axis='columns').columns.values
poly15_1 = ['x1 x8','x3 x16', 'x5 x10', 'x5 x16', 'x5', 'x5 x6', 'x3 x6', 'x3 x5', 'x5 x11', 'x3 x10', 'x3^2', 'x2^2', 'x3', 'x3 x12']
poly15_2 = poly15_2 = abs(corr_poly.Ret_PlusTwo).sort_values(ascending=False)[0:15].index
micpoly15 = MIC_poly.MIC_PlusOne.sort_values(ascending=False)[0:15].index

## Ridge

In [ ]:
#RFE Params
rfe_learner = Ridge()
rfe_params ={'alpha':100,'normalize':True,'max_iter':700,'random_state':[0]}
rfe_learner = rfe_learner.set_params(**rfe_params)
rfe_target = 'Ret_PlusTwo'

#RFE Fitting
rfe = RFE(rfe_learner,n_features_to_select=10, step=1, verbose=1)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe10 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=20, step=1, verbose=1)
rfe.fit(all_features, y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe20 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=30, step=1, verbose=0)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe30 = all_features.columns[rfe.get_support(indices=True)]

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE

#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = Ridge()
target= 'Ret_PlusTwo'
param_grid= {'alpha': [20,19,18,17,16,15,12,11,10,8,6,4,2,1,0.5],'normalize':[True],'max_iter':[700],'random_state':[0]}


#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,rfe10,rfe20,rfe30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','rfe10','rfe20','rfe30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

         ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


## Huber

In [484]:
#RFE Params
rfe_learner = HuberRegressor()
rfe_params ={'alpha':100,'max_iter':700}
rfe_learner = rfe_learner.set_params(**rfe_params)
rfe_target = 'Ret_PlusTwo'

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE

#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = HuberRegressor()
target= 'Ret_PlusTwo'
param_grid= {'alpha': [100,20,19,18,17,16,15,12,11],'epsilon':[1.35],'max_iter':[700]}

results = pd.DataFrame(columns=['algo','featureset','target','fitting_time','zerobenchmark','validationmark','mae','params'])


#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

         ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


Index(['core', 'mic10', 'mic20', 'mic30', 'corr10', 'corr20', 'corr30',
       'rfe10', 'rfe20', 'rfe30', 'polycorr15_1', 'polycorr15_2', 'micpoly15'],
      dtype='object')

'micpoly15'

'polycorr15_2'

### ExtraTreesRegressor

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)

from sklearn.ensemble import ExtraTreesRegressor


#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = ExtraTreesRegressor()
target= 'Ret_PlusOne'
param_grid= {'max_depth': [3],#tiefer
             
             'max_features': 'auto',#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_leaf': [5000], #höher
             #'min_samples_split': [],
             'n_estimators': [100,150],#mehr bedeuten dauert länger
             'n_jobs':[-1],
            'random_state':[0],
            'max_features':['sqrt']}


#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

     ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


## GBM

# Ret_Minutes

In [582]:
y['Ret_MinutesFut'] = df_train.loc[:, 'Ret_121':'Ret_180'].sum(axis=1)
y['Ret_MinutesFut_20'] = df_train.loc[:, 'Ret_121':'Ret_130'].sum(axis=1)

In [593]:
MIC_poly.head()

,MIC_PlusOne,MIC_PlusTwo,MIC_Minute
x0,0.061024,0.070398,0.089123
x1,0.065661,0.069089,0.081241
x2,0.037624,0.034081,0.048443
x3,0.041692,0.031685,0.045471
x4,0.043810,0.042430,0.063234


In [594]:
mic10 = MIC.MIC_Minute.sort_values(ascending=False)[0:10].index
mic20 = MIC.MIC_Minute.sort_values(ascending=False)[0:20].index
mic30 = MIC.MIC_Minute.sort_values(ascending=False)[0:30].index
corr10 = abs(corr_spearman.Ret_MinutesFut).sort_values(ascending=False)[0:10].index
corr20 = abs(corr_spearman.Ret_MinutesFut).sort_values(ascending=False)[0:20].index
corr30 = abs(corr_spearman.Ret_MinutesFut).sort_values(ascending=False)[0:30].index
#corrPear = abs(corr_pearson.Ret_PlusTwo).sort_values(ascending=False)[0:20].index
core=core_features.drop('Feature_7',axis='columns').columns.values
poly15_1 = ['x1 x8','x3 x16', 'x5 x10', 'x5 x16', 'x5', 'x5 x6', 'x3 x6', 'x3 x5', 'x5 x11', 'x3 x10', 'x3^2', 'x2^2', 'x3', 'x3 x12']
poly15_2 = poly15_2 = abs(corr_poly.Ret_MinutesFut).sort_values(ascending=False)[0:15].index
micpoly15 = MIC_poly.MIC_Minute.sort_values(ascending=False)[0:15].index

### Ridge

In [597]:

#RFE Params
rfe_learner = Ridge()
rfe_params ={'alpha':100,'normalize':True,'max_iter':700,'random_state':0}
rfe_learner = rfe_learner.set_params(**rfe_params)
rfe_target = 'Ret_MinutesFut'

#RFE Fitting
rfe = RFE(rfe_learner,n_features_to_select=10, step=1, verbose=0)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe10 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=20, step=1, verbose=0)
rfe.fit(all_features, y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe20 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=30, step=1, verbose=0)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe30 = all_features.columns[rfe.get_support(indices=True)]

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE

#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = Ridge()
target= 'Ret_MinutesFut'
param_grid= {'alpha': [100,50,35,20,10],'normalize':[True],'max_iter':[700]}


#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,rfe10,rfe20,rfe30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','rfe10','rfe20','rfe30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    

    for cols in grid.loc[:,'Ret_121':'Ret_180'].columns:
        
        grid[cols] = ((gs.predict(train[cols_to_use]))/60)
    
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    for cols in grid.loc[:,'Ret_121':'Ret_180'].columns:
        grid[cols] = ((gs.predict(test[cols_to_use]))/60)

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

     ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


Fitting estimator with 144 features.
Fitting estimator with 143 features.
Fitting estimator with 142 features.
Fitting estimator with 141 features.
Fitting estimator with 140 features.
Fitting estimator with 139 features.
Fitting estimator with 138 features.
Fitting estimator with 137 features.
Fitting estimator with 136 features.
Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
Fitting estimator with 127 features.
Fitting estimator with 126 features.
Fitting estimator with 125 features.
Fitting estimator with 124 features.
Fitting estimator with 123 features.
Fitting estimator with 122 features.
Fitting estimator with 121 features.
Fitting estimator with 120 features.
Fitting estimator with 119 features.
Fitting estimator with 118 features.
F

Fitting estimator with 51 features.
Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 fe

Index(['core', 'mic10', 'mic20', 'mic30', 'corr10', 'corr20', 'corr30',
       'rfe10', 'rfe20', 'rfe30', 'polycorr15_1', 'polycorr15_2', 'micpoly15'],
      dtype='object')

'core'

'mic10'

...

'rfe20'

'rfe30'

'polycorr15_1'

'polycorr15_2'

'micpoly15'

### Huber

In [618]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
from sklearn.linear_model import HuberRegressor

#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = HuberRegressor()
target= 'Ret_MinutesFut'
param_grid= {'alpha': [20,5],'max_iter':[700]}


#Choose Featuresets
featuresets = pd.DataFrame([mic10,mic20,mic30,corr10,corr20,corr30,poly15_1,poly15_2,micpoly15],index=['mic10','mic20','mic30','corr10','corr20','corr30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,31))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    

    for cols in grid.loc[:,'Ret_121':'Ret_180'].columns:
        
        grid[cols] = ((gs.predict(train[cols_to_use]))/60)
    
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    for cols in grid.loc[:,'Ret_121':'Ret_180'].columns:
        grid[cols] = ((gs.predict(test[cols_to_use]))/60)

    sample['Predicted'] = grid.values.flatten()
    #sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

     ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


Index(['mic10', 'mic20', 'mic30', 'corr10', 'corr20', 'corr30', 'polycorr15_1',
       'polycorr15_2', 'micpoly15'],
      dtype='object')

'mic10'

'mic20'

'mic30'

'corr10'

'corr20'

'corr30'

'polycorr15_1'

'polycorr15_2'

'micpoly15'

### Xtra Tree

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)

from sklearn.ensemble import ExtraTreesRegressor


#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = ExtraTreesRegressor()
target= 'Ret_MinutesFut'
param_grid= {'max_depth': [3],#tiefer
             
             'max_features': 'auto',#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_leaf': [5000], #höher
             #'min_samples_split': [],
             'n_estimators': [100,150],#mehr bedeuten dauert länger
             'n_jobs':[-1],
            'random_state':[0],
            'max_features':['sqrt']}



#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets.columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    

    for cols in grid.loc[:,'Ret_121':'Ret_180'].columns:
        
        grid[cols] = ((gs.predict(train[cols_to_use]))/60)
    
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    for cols in grid.loc[:,'Ret_121':'Ret_180'].columns:
        grid[cols] = ((gs.predict(test[cols_to_use]))/60)

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

     ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


In [619]:
results

,algo,featureset,target,fitting_time,zerobenchmark,validationmark,mae,params
0,Ridge,core,Ret_MinutesFut,6.70325,1773.92,1774.13,0.00485384,"{'alpha': 6, 'copy_X': True, 'fit_intercept': ..."
1,Ridge,mic10,Ret_MinutesFut,2.95934,1773.92,1774.12,0.00485556,"{'alpha': 20, 'copy_X': True, 'fit_intercept':..."
2,Ridge,mic20,Ret_MinutesFut,4.24575,1773.92,1774.2,0.00486075,"{'alpha': 20, 'copy_X': True, 'fit_intercept':..."
3,Ridge,mic30,Ret_MinutesFut,5.65856,1773.92,1774.24,0.00485748,"{'alpha': 20, 'copy_X': True, 'fit_intercept':..."
4,Ridge,corr10,Ret_MinutesFut,2.88769,1773.92,1774.11,0.00485946,"{'alpha': 20, 'copy_X': True, 'fit_intercept':..."
5,Ridge,corr20,Ret_MinutesFut,4.3113,1773.92,1774.12,0.00486077,"{'alpha': 20, 'copy_X': True, 'fit_intercept':..."
6,Ridge,corr30,Ret_MinutesFut,5.64229,1773.92,1774.12,0.00486032,"{'alpha': 20, 'copy_X': True, 'fit_intercept':..."
7,Ridge,rfe10,Ret_MinutesFut,2.90213,1773.92,1774.1,0.00485698,"{'alpha': 20, 'copy_X': True, 'fit_intercept':..."
8,Ridge,rfe20,Ret_MinutesFut,4.15243,1773.92,1774.11,0.00485746,"{'alpha': 20, 'copy_X': True, 'fit_intercept':..."
9,Ridge,rfe30,Ret_MinutesFut,5.97855,1773.92,1774.19,0.00486123,"{'alpha': 20, 'copy_X': True, 'fit_intercept':..."


In [522]:
from openpyxl import load_workbook
book = load_workbook('BA_Submissions/Results.xlsx')
writer = pd.ExcelWriter('BA_Submissions/Results.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
writer.sheets['Tabelle1']
startrow = writer.sheets['Tabelle1'].max_row
results.to_excel(writer, index = False,header=False,startrow=startrow,sheet_name ='Tabelle1')
writer.save()